In [59]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [60]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [61]:
data_train = pd.read_csv("/kaggle/input/ipl-2020-player-performance/Training.csv")

In [62]:
data_train['batting_points'] = data_train['Runs'] + data_train['Boundaries'] + 2* data_train['Six'] + 8* data_train['Fifty'] + 16* data_train['Hundred'] - 2* data_train['Duck']


In [63]:
data_train['bowling_points'] = 25 * data_train['Wickets'] + 8* data_train['4W_Haul'] + 16* data_train['5W_Haul'] + 8* data_train['Maidens']

In [64]:
data_train['total'] = data_train['batting_points'] + data_train['bowling_points']
print(data_train)

In [65]:
data_train.isnull().any()

In [66]:
data_train.describe()

In [67]:
data_train.shape

In [68]:
plt.figure(figsize =(10,7))
sns.scatterplot(x='batting_points', y='bowling_points', data=data_train)

In [69]:
match2020 = pd.read_csv("/kaggle/input/ipl-2020-player-performance/Matches IPL 2020.csv")

In [70]:
match2020 = match2020[['match_id', 'city', 'date']]

In [71]:
match0819 = pd.read_csv("/kaggle/input/ipl-2020-player-performance/Matches IPL 2008-2019.csv")

In [72]:
match0819 = match0819[['match_id', 'city', 'date']]

In [73]:
overall_matches = pd.merge(match0819, match2020, on='match_id', how='outer')
overall_matches

In [74]:
data_train[['match_id', 'Player_name']] = data_train['Id'].str.split('_',expand = True)


In [75]:
data_train['match_id'] = data_train['match_id'].astype(int)
data_train = pd.merge(data_train, overall_matches, on='match_id',how='inner')

In [76]:
data_train

In [77]:
data1=data_train[(data_train['Player_name']=="Bipul Sharma")]

In [78]:
data1

In [79]:
data_group = data_train.groupby(['match_id', 'Player_name'])['total'].sum()

In [80]:
data_group.reset_index()

In [81]:
player_info = data_train[['Player_name', 'match_id', 'total']]

In [82]:
player_info1= player_info.groupby(['Player_name', 'match_id']).sum()

In [83]:
player_info1.reset_index()

In [84]:
player_desc = data_train[['match_id','Player_name','Total Points']]

In [85]:
player_desc1 = player_desc.groupby(['match_id','Player_name']).sum()

In [86]:
player_desc1.reset_index()

In [87]:
from math import sqrt
rmse = sqrt(mean_squared_error(player_desc1['Total Points'], player_info1['total']))

In [88]:
rmse

In [89]:
copy_train = pd.read_csv("/kaggle/input/ipl-2020-player-performance/Training.csv")

In [90]:
X = np.array(copy_train.iloc[:,1:-1])
y = np.array(copy_train.iloc[:,-1])

In [91]:
train_x, test_x, train_y, test_y=train_test_split(X,y,test_size=0.3,random_state=1)

In [92]:
stand_scaler = StandardScaler()
stand_scaler.fit(train_x)
train_x= stand_scaler.transform(train_x)
test_x= stand_scaler.transform(test_x)

In [93]:
lr = LinearRegression()

In [94]:
lr.fit(train_x,train_y)

In [95]:
y_pred = lr.predict(test_x)

In [96]:
rmse = np.sqrt(mean_squared_error(y_pred, test_y))

In [97]:
rmse

In [98]:
submission = pd.read_csv("/kaggle/input/ipl-2020-player-performance/sample_submission.csv")

In [99]:
submission[['match_id', 'Player_name']] = submission['Id'].str.split('_', expand=True)

In [100]:
submission

In [101]:
players = submission['Player_name'].unique().tolist()

In [102]:
data2 = player_info[player_info['Player_name'].isin(players)]

In [103]:
data2

In [104]:
df= pd.DataFrame()

In [105]:
df['match_id'] = submission['match_id']

In [106]:
df['Player_name'] = submission['Player_name']

In [107]:
df['Total Points'] = player_desc['Total Points']

In [108]:
df

In [109]:
df.isnull().any()

In [110]:
df['Id'] = df['match_id']+'_'+df['Player_name']

In [111]:
df

In [112]:
final_subm=pd.DataFrame()

In [113]:
final_subm['Id'] = df['Id']

In [114]:
final_subm['Total Points']=df['Total Points']

In [115]:
final_subm

In [116]:
final_subm.to_csv('My_Submission.csv', index=False)